<a href="https://colab.research.google.com/github/Ramki13084/tensorflow_sample/blob/main/Transformer_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn

In [2]:
class SelfAttention(nn.Module):
      def __init__(self,embed_size,heads):
          super(SelfAttention,self).__init__()
          self.embed_size=embed_size
          self.heads = heads
          self.head_dim = embed_size//heads
          assert(self.head_dim*self.heads ==self.embed_size)
          self.values = nn.Linear(self.head_dim,self.head_dim,bias=False)
          self.keys = nn.Linear(self.head_dim,self.head_dim,bias=False)
          self.queries = nn.Linear(self.head_dim,self.head_dim,bias=False)
          self.fc_out =  nn.Linear(heads*self.head_dim,embed_size)
      def forward(self,values,keys,queries,mask):
          N = query.shape[0]
          value_len,key_len,query_len = values.shape[1],keys.shape[1],query.shape[1]  